# 1 - Setup<a name="1"></a>

In [1]:
import sys
import os

In [2]:
os.environ['HADOOP_HOME'] = r"C:\spark-3.1.3-bin-hadoop3.2"
sys.path.append(r"C:\spark-3.1.3-bin-hadoop3.2")

os.environ['SPARK_HOME'] = r"C:\spark-3.1.3-bin-hadoop3.2"
sys.path.append(r"C:\spark-3.1.3-bin-hadoop3.2")

os.environ['JAVA_HOME'] = r"C:\Program Files\Java\jdk1.8.0_351"
sys.path.append(r"C:\Program Files\Java\jdk1.8.0_351")

os.environ['CLASSPATH'] = r"C:\Program Files\Java\jdk1.8.0_351"
sys.path.append(r"C:\Program Files\Java\jdk1.8.0_351")

In [3]:
os.environ['HADOOP_HOME'] = r"C:\spark-3.1.3-bin-hadoop3.2"
sys.path.append(r"C:\spark-3.1.3-bin-hadoop3.2")

os.environ['SPARK_HOME'] = r"C:\spark-3.1.3-bin-hadoop3.2"
sys.path.append(r"C:\spark-3.1.3-bin-hadoop3.2")

os.environ['JAVA_HOME'] = r"C:\Program Files\Java\jdk1.8.0_341"
sys.path.append(r"C:\Program Files\Java\jdk1.8.0_341")

os.environ['CLASSPATH'] = r"C:\Program Files\Java\jdk1.8.0_341"
sys.path.append(r"C:\Program Files\Java\jdk1.8.0_341")

In [3]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext, sql

from pyspark.sql.functions import array_contains, col, explode, regexp_replace, when, substring, length, expr, arrays_zip
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("TF-IDF").getOrCreate()
sc = SparkContext.getOrCreate()

## 2 - Load data<a name="2"></a>

In [5]:
# queries_data = spark.read.option("multiline", "true").json(sys.argv[1])

# candidate_data = spark.read.option("multiline", "true").json(sys.argv[2])

# with open(sys.argv[3], "r") as stopword_file:
#     stopwords = stopword_file.read().splitlines()

In [6]:
queries_data = spark.read.option("multiline", "true").json("query.json")
#query_data = spark.read.option("multiline", "true").json(sys.argv[1])

In [7]:
queries_data.show()

+--------------------+---------+
|            abstract|       id|
+--------------------+---------+
|  A fully differe...|0704.0001|
|  Partial cubes a...|0704.0010|
|  We study the tw...|0704.0006|
|  A rather non-st...|0704.0007|
|  A general formu...|0704.0008|
|  We discuss the ...|0704.0009|
|  Recently, Bruin...|0704.0012|
|  We study space-...|0704.0052|
|  We investigate ...|0704.0030|
|  In 2nd order ca...|0704.0134|
|  We give a quant...|0704.0086|
|  Sparse Code Div...|0704.0098|
|  We consider a s...|0704.0117|
|  We study the ps...|0704.0324|
|  A nonperturbati...|0704.0142|
|  There is a huge...|0704.1267|
|  We have analyze...|0901.0031|
|  Magnetic fields...|0901.0035|
|  The magnetic fi...|0901.0040|
|  We consider Bra...|0901.0173|
+--------------------+---------+



In [8]:
candidate_data = spark.read.option("multiline", "true").json("candidate.json")
#candidate_data = spark.read.option("multiline", "true").json(sys.argv[2])

In [9]:
candidate_data.show(1, truncate = False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|abstract                                                                                                                                                                                                     

In [10]:
with open("stopwords.txt", "r") as stopword_file:
#with open(sys.argv[3], "r") as stopword_file:
    stopwords = stopword_file.read().splitlines()

## 3 - Preprocess data <a name="3"></a>

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, CountVectorizer, Tokenizer, HashingTF, IDF, StopWordsRemover
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import DenseVector
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf, col
import nltk
import re
import math

In [42]:
def remove_brackets(input_str):
    return re.sub(r'[()]', '', input_str)

def custom_tokenizer(text):
    # Remove spaces and other delimiters
    tokens = re.split(r'\s+|[,;.!?\-]+', text)
    # Remove stopwords and convert to lowercase
    filtered_tokens = [remove_brackets(token.lower()) for token in tokens if token.lower() not in stopwords]
    return filtered_tokens

preprocess_udf = udf(custom_tokenizer, ArrayType(StringType()))

In [43]:
def unpack_words(doc):
    doc, doc_id, words = doc
    return [(doc_id, words[i])for i in range(len(words)) if words[i] != '']

#input --> (q_id, [q_terms]), (c_id, [c_terms])
def get_normed_TFIDF(q_rdd, c_rdd):
    
    # (doc_id, term)
    c_words_rdd = c_rdd.flatMap(unpack_words)
    q_words_rdd = q_rdd.flatMap(unpack_words)
    
    #count TF --> ((c_id, term), 1)
    c_TF_rdd = c_words_rdd.map(lambda x : (((x[0]),x[1]),1) ) 
    # ((c_id, term), c_TF)
    c_TF_rdd = c_TF_rdd.reduceByKey(lambda a, b: a + b)
    #count TF --> ((term, q_id), 1)
    q_TF_rdd = q_words_rdd.map(lambda x : (((x[1]),x[0]),1) ) 
    # ((term, q_id), q_TF)
    q_TF_rdd = q_TF_rdd.reduceByKey(lambda a, b: a + b)
    
    # count N (number of abstract)
    N = c_rdd.count()
    
    # count c_DF
    c_DF_rdd = c_TF_rdd.map(lambda x: (x[0][1],1) ) # (term, 1)
    # (term, c_DF)
    c_DF_rdd = c_DF_rdd.reduceByKey(lambda a, b: a + b)
    # calculate qc_DF --> (q_id, term) --> ((term, q_id), q_TF) + (term, c_DF) --> ((term, q_id), qc_DF)
    qc_DF_rdd = q_TF_rdd.map(lambda x: (x[0][0], x[0][1]) ).leftOuterJoin(c_DF_rdd)\
                           .map(lambda x: ((x[0], x[1][0]), 1 if x[1][1] is None else x[1][1]+1) ) 
    # calculate cq_DF --> (c_id, term) --> (term, q_id) + (term, c_DF) --> (term, cq_DF))
    cq_DF_rdd = q_TF_rdd.map(lambda x: (x[0][0], x[0][1]) ).rightOuterJoin(c_DF_rdd)\
                           .map(lambda x: (x[0], x[1][1] if x[1][0] is None else x[1][1]+1) ) 
    

    # ((term, q_id), qc_IDF))
    q_IDF_rdd = qc_DF_rdd.map(lambda x: (x[0], math.log10((N+1)/(x[1]))) )
    # (term, cq_IDF))
    c_IDF_rdd = cq_DF_rdd.map(lambda x: (x[0], math.log10((N+1)/(x[1]))) )                            
    
    # Calculate c_TF-IDF --> (term,((c_id, TF), IDF))
    c_TFIDF_rdd = c_TF_rdd.map(lambda x: (x[0][1], (x[0][0], x[1])) ).join(c_IDF_rdd) 
    # (c_id, (term,c_TF-IDF))
    c_TFIDF_rdd = c_TFIDF_rdd.map(lambda x: (x[1][0][0], (x[0],(1+math.log10(x[1][0][1]))*x[1][1])) )
                                
    # Calculate q_TF-IDF --> ((term, q_id), q_TF) + ((term, q_id), q_IDF)) --> ((term, q_id), (q_TF, q_IDF))
    q_TFIDF_rdd = q_TF_rdd.join(q_IDF_rdd)
    # (q_id, (term,q_TF-IDF))                            
    q_TFIDF_rdd = q_TFIDF_rdd.map(lambda x : (x[0][1], (x[0][0],(1+math.log10(x[1][0]))*x[1][1])) )

                                
    # calculate sqrt-sum-squared TF-IDF per document 
    # (doc id, TF-IDF**2) --> (doc id, sum(TF-IDF**2))
    c_sum_sq_TFIDF_rdd = c_TFIDF_rdd.map(lambda x: (x[0], x[1][1]**2) ).reduceByKey(lambda a, b: a + b)
    q_sum_sq_TFIDF_rdd = q_TFIDF_rdd.map(lambda x: (x[0], x[1][1]**2) ).reduceByKey(lambda a, b: a + b)                            
    # (doc id, sum(TF-IDF**2)) --> (doc id, sqrt(sum(TF-IDF**2)))
    c_sum_sqrt_TFIDF_rdd = c_sum_sq_TFIDF_rdd.map(lambda x: (x[0], math.sqrt(x[1])))
    q_sum_sqrt_TFIDF_rdd = q_sum_sq_TFIDF_rdd.map(lambda x: (x[0], math.sqrt(x[1])))
    # calculate normalized TF-IDF --> (doc_id, (term, normed_TF-IDF))
    c_norm_TFIDF_rdd = c_TFIDF_rdd.join(c_sum_sqrt_TFIDF_rdd)\
                        .map(lambda x : (x[0],(x[1][0][0], x[1][0][1]/x[1][1])) ) 
    q_norm_TFIDF_rdd = q_TFIDF_rdd.join(q_sum_sqrt_TFIDF_rdd)\
                        .map(lambda x : (x[0],(x[1][0][0], x[1][0][1]/x[1][1])) ) 
                                
    return q_norm_TFIDF_rdd, c_norm_TFIDF_rdd

In [44]:
# apply preprocessing to query and candidate data
queries_data = queries_data.withColumn("words", preprocess_udf(queries_data["abstract"]))
candidate_data = candidate_data.withColumn("words", preprocess_udf(candidate_data["abstract"]))

In [45]:
rows = queries_data.collect()
for row in [rows[0]]:
    # Create a new DataFrame containing only the current row
    query_data = spark.createDataFrame([row], schema=queries_data.schema)

In [46]:
query_rdd = query_data.rdd
candidate_rdd = candidate_data.rdd
q_norm_TFIDF_rdd, c_norm_TFIDF_rdd = get_normed_TFIDF(query_rdd, candidate_rdd)

In [47]:
q_norm_TFIDF_rdd.collect()

[('0704.0001', ('hadron', 0.1553789501100955)),
 ('0704.0001', ('subprocesses', 0.18105297298989445)),
 ('0704.0001', ('valid', 0.11704343918321557)),
 ('0704.0001', ('quark', 0.10254020495862)),
 ('0704.0001', ('contrasted', 0.17042645998047076)),
 ('0704.0001', ('showing', 0.10254020495862)),
 ('0704.0001', ('enhanced', 0.11007983592649924)),
 ('0704.0001', ('cdf', 0.17042645998047076)),
 ('0704.0001', ('differential', 0.08941578313936849)),
 ('0704.0001', ('processes', 0.07418803238810681)),
 ('0704.0001', ('phase', 0.062121145627368646)),
 ('0704.0001', ('region', 0.07008463821918597)),
 ('0704.0001', ('diphoton', 0.25919004688401504)),
 ('0704.0001', ('judicious', 0.1992191169671974)),
 ('0704.0001', ('orders', 0.10320373928138935)),
 ('0704.0001', ('tests', 0.12655454105798564)),
 ('0704.0001', ('quantum', 0.057506846232717006)),
 ('0704.0001', ('logarithmic', 0.14163380299374412)),
 ('0704.0001', ('space', 0.057624636120106904)),
 ('0704.0001', ('calculation', 0.1301580281246778

In [36]:
#cdf = spark.createDataFrame(c_norm_TFIDF_rdd.map(lambda x : (x[0], x[1][0], x[1][1])), ["c_id", "terms", "tfidf"])

In [39]:
#cdf.filter(cdf["c_id"] == "2301.04294").show(100)

+----------+------------+-------------------+
|      c_id|       terms|              tfidf|
+----------+------------+-------------------+
|2301.04294|        work| 0.1155173860178222|
|2301.04294|        poor|0.20349064697571326|
|2301.04294|       higgs|0.21349692649701865|
|2301.04294| logarithmic| 0.2531941520360997|
|2301.04294|perturbative|0.21131739124846274|
|2301.04294|    accurate|0.19092206103003956|
|2301.04294|      decays|0.21349692649701865|
|2301.04294|    function|0.09597273596332617|
|2301.04294|     channel|0.15512283570225957|
|2301.04294| resummation|0.28073287353696663|
|2301.04294|       three|0.11055352998393433|
|2301.04294|      extend|0.17778947702388231|
|2301.04294|   functions|0.12737066709501602|
|2301.04294|        loop| 0.1967861493835199|
|2301.04294|       order|0.09366954912857464|
|2301.04294|      values|0.13899327076924298|
|2301.04294| convergence|0.18824410024229055|
|2301.04294|      thrust|0.35613754854918395|
|2301.04294|distribution| 0.115517

In [17]:
# calculate L2 norm
def get_L2norm(rdd):
    # map to (doc_id, tfidf^2), then reduce by key:summation
    sum_sq_rdd = rdd.map(lambda x: (x[0], x[1][1]**2) ).reduceByKey(lambda a, b: a + b)
    # map to (doc_id, sqrt(sum of squares))
    L2norm_rdd = sum_sq_rdd.map(lambda x: (x[0], math.sqrt(x[1])))
    return L2norm_rdd

In [18]:
#input --> (term, ((q_id, q_tfidf),(c_id,c_tfidf))), output --> (c_id, ((q_id,relevance,q), c))
def get_relevance(merged_TFIDF_rdd):
    # map to ((q_id,c_id),(q_tfidf*c_tfidf))
    product_TFIDF_rdd = merged_TFIDF_rdd.map(lambda x: ((x[1][0][0],x[1][1][0]),(x[1][0][1]*x[1][1][1])) )
    # reduce by key to get cosine similarity score, L2norm is 1, so division by L2_norm is trivial
    abstract_relevance_rdd = product_TFIDF_rdd.reduceByKey(lambda a, b: a + b)
    # get most relevant pairs for each query --> (q_id, (c_id, cos_sim))
    matching_abstract_rdd = abstract_relevance_rdd.map(lambda x : (x[0][0], (x[0][1], x[1])) )
    matching_abstract_rdd = matching_abstract_rdd.reduceByKey(lambda a, b: a if a[1] > b[1] else b)
    # merge abstracts --> (q_id, ((c_id,abs_relevance),q_abs))
    merged_result_rdd = matching_abstract_rdd.join(query_data.select("id","abstract").rdd)
    # map from (q_id, ((c_id,abs_relevance),q_abs)) to (c_id, (q_id,relevance,q_abs)) 
    # join to candidate abstract --> (c_id, ((q_id,relevance,q_abs), c_abs))
    merged_result_rdd = merged_result_rdd.map(lambda x : (x[1][0][0], (x[0],x[1][0][1],x[1][1])) )\
                    .join(candidate_data.select("id","abstract").rdd) 
    return merged_result_rdd

In [19]:
# calulate cosine similarity
# map to (term, (q_id,tfidf)) then merge on terms --> (term, ((q_id, q_tfidf),(c_id,c_tfidf)))
merged_TFIDF_rdd = q_norm_TFIDF_rdd.map(lambda x: (x[1][0], (x[0],x[1][1])) )\
                .join(c_norm_TFIDF_rdd.map(lambda x: (x[1][0], (x[0],x[1][1])) )) 
# output --> (c_id, ((q_id,relevance,q_abs), c_abs))
merged_result_rdd = get_relevance(merged_TFIDF_rdd)

In [20]:
merged_TFIDF_rdd.collect()

[('hadron',
  (('0704.0001', 0.1529307420984682),
   ('nucl-th/9509015', 0.10316534757607417))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('1505.04110', 0.12952308486925104))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('0904.0937', 0.07235180212583271))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('1809.09127', 0.11484190577575583))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('1211.5897', 0.1403556348591433))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('1510.05692', 0.08436455437546228))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('1111.2830', 0.1195359501094894))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('1503.02197', 0.08748078668206731))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('1001.1595', 0.13450311641220142))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('2205.12641', 0.10001396667900156))),
 ('hadron',
  (('0704.0001', 0.1529307420984682), ('2002.00616', 0.12784599511200248))),
 ('hadron',
  (('07

In [18]:
def split_sent(block):
    sentences = nltk.sent_tokenize(block)
    return sentences

In [19]:
# merged_result --> (c_id, ((q_id,relevance,q_abs), c_abs)) 
# split c_abstract to list of sentence --> (q_id, (c_id, relevance, q_abs), [c_sentences])
split_sent_rdd = merged_result_rdd.map(lambda x : (x[1][0][0], (x[0], x[1][0][1], x[1][0][2]), split_sent(x[1][1])) ) 

In [20]:
def unpack_sentences(doc):
    q_id, (c_id, relevance, q_abs), c_sentences = doc
    return [(q_id, (c_id, relevance, q_abs), c_sentences[i]) for i in range(len(c_sentences))]

# flatmap list of sentence into one row per sentence
flat_sent_rdd = split_sent_rdd.flatMap(unpack_sentences) #(q_id, (c_id, relevance, q_abs), c_sentence)

In [21]:
def unpack_words_sent(doc):
    q_id, c_sentence, words = doc
    return [(q_id, c_sentence, words[i])for i in range(len(words)) if words[i] != '']

#input --> (q_id, (c_id, relevance, q_abs), c_sentence) 
def get_normed_TFIDF_sent(flat_sent_rdd):    
    # transform (q_id, (c_id, relevance, q_abs), c_sentence) to (q_id, c_sentence, [words])
    words_sent_rdd = flat_sent_rdd.map(lambda x : (x[0], x[2], custom_tokenizer(x[2])) )
    # flatten into (q_id, c_sentence, words)
    flat_words_sent_rdd = words_sent_rdd.flatMap(unpack_words_sent) # words
    # calculate TF
    TF_rdd = flat_words_sent_rdd.map(lambda x : (((x[0]),x[1],x[2]),1) ) # ((doc_id,sent,term), 1)
    TF_rdd = TF_rdd.reduceByKey(lambda a, b: a + b)
    #count DF (group by doc_id, term) --> ((doc_id, term), 1)
    DF_rdd = TF_rdd.map(lambda x: ((x[0][0],x[0][2]),1) ) 
    # ((doc_id, term), DF) --> (doc_id, (term, DF)) 
    DF_rdd = DF_rdd.reduceByKey(lambda a, b: a + b).map(lambda x : (x[0][0], (x[0][1], x[1]))) 
    #total number of sentences in each abstract
    #groupby number of query_id because each row have each sentence, groupby query give number of sentence in each query
    N_doc_rdd = flat_sent_rdd.map(lambda x: ((x[0]), 1)).reduceByKey(lambda a, b: a + b)
    DF_rdd_merge = DF_rdd.join(N_doc_rdd) # (doc_id, ((term, DF), N))
    # calculate IDF
    # ((doc_id,sent,term), TF) --> ((doc_id,term), (sent,TF)) ; (doc_id, ((term, DF), N)) --> ((doc_id,term), (DF,N))
    TF_rdd_merge = TF_rdd.map(lambda x: ((x[0][0],x[0][2]),(x[0][1],x[1])) )\
                .join(DF_rdd_merge.map(lambda x: ((x[0],x[1][0][0]),(x[1][0][1],x[1][1])) ))
    # ((doc_id,term), ((sent,TF),(DF,N))) --> ((doc_id, sent), (term, TF-IDF))
    TFIDF_rdd = TF_rdd_merge.map(lambda x: ((x[0][0], x[1][0][0]), (x[0][1], (1+math.log10(x[1][0][1]))*math.log10(x[1][1][1]/x[1][1][0]))) )
    # calculate sqrt-sum-squared TF-IDF per document
    sum_sq_TFIDF_rdd = TFIDF_rdd.map(lambda x: (x[0], x[1][1]**2) ).reduceByKey(lambda a, b: a + b)
    # ((doc_id, sent), sqrt(S))
    sum_sqrt_TFIDF_rdd = sum_sq_TFIDF_rdd.map(lambda x: (x[0], math.sqrt(x[1])) )
    # calculate normalized tfidf --> ((doc_id,sent),((term, TF-IDF), sqrt(S))) --> ((doc_id,sent), (term, norm_tfidf))
    norm_TFIDF_rdd = TFIDF_rdd.join(sum_sqrt_TFIDF_rdd).map(lambda x : (x[0],(x[1][0][0], x[1][0][1]/x[1][1])) )
    return norm_TFIDF_rdd

In [22]:
# input --> (q_id, (c_id, relevance, q_abs), c_sentence), output--> ((doc_id,sent), (term, norm_tfidf))
sent_norm_TFIDF_rdd = get_normed_TFIDF_sent(flat_sent_rdd)

In [23]:
sent_norm_TFIDF_rdd.collect()

[(('0901.0173',
   '  A cosmological model where a fermion field is non-minimally coupled with the\ngravitational field is studied.'),
  ('minimally', 0.19990265386264808)),
 (('0901.0173',
   '  A cosmological model where a fermion field is non-minimally coupled with the\ngravitational field is studied.'),
  ('cosmological', 0.19990265386264808)),
 (('0901.0173',
   '  A cosmological model where a fermion field is non-minimally coupled with the\ngravitational field is studied.'),
  ('gravitational', 0.5416384978119665)),
 (('0901.0173',
   '  A cosmological model where a fermion field is non-minimally coupled with the\ngravitational field is studied.'),
  ('field', 0.0)),
 (('0901.0173',
   '  A cosmological model where a fermion field is non-minimally coupled with the\ngravitational field is studied.'),
  ('studied', 0.5416384978119665)),
 (('0901.0173',
   '  A cosmological model where a fermion field is non-minimally coupled with the\ngravitational field is studied.'),
  ('fermion'

In [24]:
# merge with <q_norm_TFIDF_rdd> on doc_id, term   
#(q_id, (term, abs_tfidf)) --> ((q_id, terms), (abs_tfidf, (sent, sent_tfidf))
merged_sent_rdd = q_norm_TFIDF_rdd.map(lambda x: ((x[0], x[1][0]), x[1][1]))\
                                .join(sent_norm_TFIDF_rdd.map(lambda x: ((x[0][0],x[1][0]),(x[0][1], x[1][1])) ))
# map to (term, ((doc_id, doc_tfidf),(sent,sent_tfidf)))
merged_sent_TFIDF_rdd = merged_sent_rdd.map(lambda x : (x[0][1], ((x[0][0],x[1][0]),(x[1][1][0],x[1][1][1]))) )

In [25]:
#input --> (term, ((doc_id, doc_tfidf),(sent,sent_tfidf))), output --> (sent, ((doc_id,relevance,q), c))
def get_relevance_sent(merged_TFIDF_rdd):
    # map to ((doc_id,sent),(doc_tfidf*sent_tfidf))
    product_TFIDF_rdd = merged_TFIDF_rdd.map(lambda x: ((x[1][0][0],x[1][1][0]),(x[1][0][1]*x[1][1][1])) )
    # reduce by key, L2norm is 1 --> ((doc_id,sent), sum(doc_tfidf*sent_tfidf))
    abstract_relevance_rdd = product_TFIDF_rdd.reduceByKey(lambda a, b: a + b)
    # get most relevant pairs for each query
    # map to (doc_id, (sent, sum(doc_tfidf*sent_tfidf)))
    matching_abstract_rdd = abstract_relevance_rdd.map(lambda x : (x[0][0], (x[0][1], x[1])) )
    # reduce to (doc_id, (sent, max(doc_tfidf*sent_tfidf)))
    matching_abstract_rdd = matching_abstract_rdd.reduceByKey(lambda a, b: a if a[1] > b[1] else b)
    # merge abstracts to split_sent_rdd: (q_id, (c_id, relevance, q_abs), [c_sentences]) --> (q_id, (c_id, relevance)
    # --> (q_id, ((sent, sent_relevance),(c_id, relevance)) )
    # --> (q_id, c_id, relevance, sent, sent_relevance)
    merged_result_rdd = matching_abstract_rdd.join(split_sent_rdd.map(lambda x: (x[0], (x[1][0],x[1][1]))))\
                        .map(lambda x : (x[0],x[1][1][0],x[1][1][1], x[1][0][0], x[1][0][1]))
    return merged_result_rdd

In [26]:
# output --> (q_id, c_id, relevance, sent, sent_relevance)
merged_sent_result_rdd = get_relevance_sent(merged_sent_TFIDF_rdd)

In [ ]:
#merged_sent_result_rdd.coalesce(1).saveAsTextFile(sys.argv[4])

In [28]:
merged_sent_result_rdd.collect()

[('0901.0173',
  '0807.1965',
  0.11534856550986425,
  '  A cosmological model where a fermion field is non-minimally coupled with the\ngravitational field is studied.',
  0.12644026280589327)]

In [ ]:
# Collect the RDD into a list with one item
result_list = merged_sent_result_rdd.collect()

# Create a storage list and append the item to it
storage_list = []
storage_list.extend(result_list)

# Output the storage list to a text file
output_file_path = "output.txt"
with open(output_file_path, 'w') as output_file:
    for item in storage_list:
        output_file.write(str(item) + '\n')


## Method 1

In [ ]:
# Apply hashing to produce a fixed number of output indices, to reduce dimensionality to save memory
# tradeoff is hash collision (information loss, the lower the numFeatures, the more information loss)
size = 262144
hashingTF = HashingTF(inputCol="words", outputCol="tf_features", numFeatures= size)

#the tf_features output is in the form: (numFeatures,[a1,a2,a3,a4],[b1,b2,b3,b4])
# where "a" is the hash-index and "b" is frequency of that index
query_featurised = hashingTF.transform(query_data)
candidate_featurised = hashingTF.transform(candidate_data)

In [ ]:
candidate_featurised.show()

+--------------------+----------+--------------------+--------------------+
|            abstract|        id|               words|         tf_features|
+--------------------+----------+--------------------+--------------------+
|  The $3x+1$ Prob...|1504.03040|[, $3x+1$, proble...|(262144,[9812,125...|
|  We characterize...| 1203.6290|[, characterize, ...|(262144,[645,1252...|
|  This paper prop...|1806.10447|[, paper, propose...|(262144,[3564,163...|
|  We develop the ...| 1204.0126|[, develop, repre...|(262144,[2548,148...|
|  We present a ca...| 0707.3149|[, cascade, model...|(262144,[916,6211...|
|  In an earlier p...|1709.07717|[, earlier, paper...|(262144,[2931,158...|
|  We consider the...| 1410.0511|[, consider, clas...|(262144,[7453,882...|
|  A perfect struc...|1906.10430|[, perfect, struc...|(262144,[6973,699...|
|  Proton-rich nuc...|2104.14093|[, proton, rich, ...|(262144,[521,706,...|
|  We present a si...|2103.02188|[, simple, powerf...|(262144,[6715,980...|
|  In this i

## 4 - Calculate normalized TF-IDF

In [ ]:
def unpack_vector(doc):
    doc, doc_id, vector = doc
    num_features = vector.size
    term_indices = vector.indices.tolist()
    term_frequencies = vector.values.tolist()
    return [(doc_id, term_indices[i], term_frequencies[i]) for i in range(len(term_indices))]

def vectorize(values, index):
    return Vectors.sparse(size, index, values)
vectorize_tfidf_udf = udf(vectorize, VectorUDT())


def get_tfidfvector(df):
    rdd = df.rdd
    #stack each term --> one document will have many rows with one term per row (equivalent to melt in pd)
    tf_rdd = rdd.flatMap(unpack_vector)
    tf_df = tf_rdd.toDF(["doc_id","term_index", "TF"])

    # get document with term frequnecy
    df_rdd = tf_rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b)

    # get IDF
    N = rdd.count()
    idf_rdd = df_rdd.map(lambda x: (x[0], math.log10(N/(x[1]))))
    idf_df = idf_rdd.toDF(["term_index_1", "IDF"])

    # join TF and IDF dataframe
    tfidf_df = tf_df.join(idf_df, tf_df.term_index == idf_df.term_index_1, "inner")
    # calculate TF-IDF sum square per document
    tfidf_df = tfidf_df.withColumn("TFIDF_raw_Square", (((1+log10(col("TF"))))*col("IDF")) ** 2)
    window_spec = Window.partitionBy("doc_id")
    tfidf_df = tfidf_df.withColumn("SumOfSquares", sum(col("TFIDF_raw_Square")).over(window_spec))
    # calculate normalized TF-IDF
    tfidf_df = tfidf_df.withColumn("TFIDF", (((1+log10(col("TF"))))*col("IDF"))/col("SumOfSquares"))\
    .select("doc_id","term_index","TFIDF")

    # group tfidf and terms_index by doc_id (pivot by doc id, then concat columns(term_index) into list per doc)
    grouped_df_tfidf = tfidf_df.groupBy("doc_id").agg(collect_list("TFIDF").alias("TFIDF_List"))
    grouped_df_termID = tfidf_df.groupBy("doc_id").agg(collect_list("term_index").alias("ID_List")).withColumnRenamed("doc_id", "doc_id_1")
    grouped_df = grouped_df_tfidf.join(grouped_df_termID, grouped_df_tfidf.doc_id == grouped_df_termID.doc_id_1, "inner").select("doc_id","TFIDF_List","ID_List")
    # sort tfidf and term_indices for vectorization
    sorted_df = grouped_df.withColumn("Combined_List", arrays_zip(col("ID_List"), col("TFIDF_List"))) \
             .withColumn("Sorted_List", sort_array(col("Combined_List"), asc=True)) \
             .withColumn("Sorted_ID_List", expr("transform(Sorted_List, x -> x.ID_List)")) \
             .withColumn("Sorted_TFIDF_List", expr("transform(Sorted_List, x -> x.TFIDF_List)")) \
             .select("doc_id", "Sorted_ID_List", "Sorted_TFIDF_List")
    # vectorize
    sorted_df = sorted_df.withColumn("TFIDF_Vector", vectorize_tfidf_udf(sorted_df["Sorted_TFIDF_List"],sorted_df["Sorted_ID_List"]))\
        .select("doc_id", "TFIDF_Vector")

    return sorted_df

In [ ]:
query_tfidf = get_tfidfvector(query_featurised.select("abstract","id","tf_features"))
candidate_tfidf = get_tfidfvector(candidate_featurised.select("abstract","id","tf_features"))

In [ ]:
query_tfidf = spark.createDataFrame(query_tfidf.collect())
candidate_tfidf = spark.createDataFrame(candidate_tfidf.collect())

## 5 - Calculate relevance by Cosine Similarity

In [ ]:
def cosine_sim(v1, v2):
    dot_product = float(v1.dot(v2))
    #L2 Norm -> sum of squares
    L2norm_v1 = float(v1.norm(2))
    L2norm_v2 = float(v2.norm(2))
    cos_sim = dot_product / (L2norm_v1 * L2norm_v2)
    return cos_sim

cos_sim_udf = udf(cosine_sim, DoubleType())

In [ ]:
# rename columns before cross join
query_tfidf = query_tfidf.withColumnRenamed("doc_id","query_id")\
    .withColumnRenamed("TFIDF_Vector","query_TFIDF_Vector")
candidate_tfidf = candidate_tfidf.withColumnRenamed("doc_id","candidate_id")\
    .withColumnRenamed("TFIDF_Vector","candidate_TFIDF_Vector")

# Calculate cosine similarity for each combination of query and candidate documents
result_data = query_tfidf.crossJoin(candidate_tfidf)\
.withColumn("abstract_relevance_score", cos_sim_udf("query_TFIDF_Vector", "candidate_TFIDF_Vector"))\
.select("query_id", "candidate_id", "abstract_relevance_score")


In [ ]:
# remove rows joining to themselves
result_data = result_data.filter(col("query_id") != col("candidate_id"))

# Get candidate abstract with highest relevance to query abstract for each query abstract
# group by query id and order by cos_sim
groupbyID = Window.partitionBy("query_id").orderBy(F.desc("abstract_relevance_score"))
grouped_result = result_data.withColumn("grouped_index", F.row_number().over(groupbyID))
top_result = grouped_result.filter(F.col("grouped_index") == 1).drop("grouped_index")

In [ ]:
top_result = spark.createDataFrame(top_result.collect())

In [ ]:
result_df = top_result.join(query_data.select("abstract","id")\
                            .withColumnRenamed("id", "query_id")\
                            .withColumnRenamed("abstract", "query_abstract")\
                            ,on = ["query_id"], how = "inner")
result_df = result_df.join(candidate_data.select("abstract","id")\
                           .withColumnRenamed("id", "candidate_id")\
                           .withColumnRenamed("abstract", "candidate_abstract")\
                           ,on = ["candidate_id"], how = "inner")
result_df = spark.createDataFrame(result_df.collect())

## For each abstract pair, get most matching sentence pair

## 6 - preprocess to stack each sentence in abstract into each row

In [ ]:
def split_sent(block):
    sentences = nltk.sent_tokenize(block)
    return sentences
split_sent_udf = udf(split_sent, ArrayType(StringType()))

In [ ]:
# split into sentences
result_df = result_df.withColumn("candidate_sentences", split_sent_udf("candidate_abstract"))

In [ ]:
result_df.select("candidate_id",\
                 "query_id",\
                 "abstract_relevance_score",\
                 "candidate_sentences").show()

+----------------+---------+------------------------+--------------------+
|    candidate_id| query_id|abstract_relevance_score| candidate_sentences|
+----------------+---------+------------------------+--------------------+
|      1507.07518|0704.0006|        0.10591162797933|[  The electron s...|
|cond-mat/0611254|0704.0117|     0.16276377377965698|[  Phase diagram ...|
|       0712.1113|0704.0142|     0.10537048296383945|[  The structure ...|
|      1910.00059|0704.0324|     0.11356998948185043|[  We present nec...|
|      1602.04317|0704.0012|     0.12158629441450435|[  We prove that ...|
|      1612.06457|0704.1267|       0.098235446735887|[  Multispectral ...|
|      2107.04550|0704.0030|     0.12092555295496764|[  Electronic fla...|
|       1101.4311|0901.0173|     0.14245825822034786|[  For the tensor...|
|       1303.4880|0901.0031|     0.12481041882853254|[  Using RXTE obs...|
|       0906.3837|0704.0007|     0.15119264778769956|[  We calculate a...|
|   gr-qc/0212108|0704.01

In [ ]:
def unpack_sentences(df):
    candidate_id, query_id, abstract_relevance_score, sentences = df
    return [(candidate_id, query_id, abstract_relevance_score\
             ,sentences[i]) for i in range(len(sentences))]

In [ ]:
candidate_result_rdd = result_df.select("candidate_id",\
                             "query_id",\
                             "abstract_relevance_score",\
                             "candidate_sentences").rdd

#stack each term --> one document will have many rows with one term per row (equivalent to melt in pd)
candidate_result_flat_rdd = candidate_result_rdd.flatMap(unpack_sentences)

candidate_sentences_df = candidate_result_flat_rdd.toDF(["candidate_id",\
                                     "query_id",\
                                     "abstract_relevance_score",\
                                     "candidate_sentences"])

In [ ]:
#split sentences into words
candidate_sentences_df = candidate_sentences_df.withColumn("words", preprocess_udf(candidate_sentences_df["candidate_sentences"]))

In [ ]:
candidate_sentences_df = spark.createDataFrame(candidate_sentences_df.collect())

In [ ]:
#the tf_features output is in the form: (numFeatures,[a1,a2,a3,a4],[b1,b2,b3,b4])
# where "a" is the hash-index and "b" is frequency of that index
candidate_sent_featurised = hashingTF.transform(candidate_sentences_df)

In [ ]:
candidate_sent_featurised = spark.createDataFrame(candidate_sent_featurised.collect())

In [ ]:
merged_df = candidate_sent_featurised.join(query_tfidf,on = ["query_id"], how = "inner")\
                                     .select("query_id","candidate_id","abstract_relevance_score","candidate_sentences","tf_features","query_TFIDF_Vector")

In [ ]:
merged_df = spark.createDataFrame(merged_df.collect())

## 7 - Calculate TF-IDF of each sentence

In [ ]:
def unpack_sent_vector(doc):
    query_id, doc_id, vector = doc
    term_indices = vector.indices.tolist()
    term_frequencies = vector.values.tolist()
    # query_id, sentence, hash_indices, term_frequencies
    return [(query_id, doc_id, term_indices[i], term_frequencies[i]) for i in range(len(term_indices))]

def vectorize(values, index):
    return Vectors.sparse(size, index, values)
vectorize_tfidf_udf = udf(vectorize, VectorUDT())


#doc_id is the sentence
def get_sent_tfidfvector(df):
    rdd = df.rdd
    #stack each term --> one document will have many rows with one term per row (equivalent to melt in pd)
    tf_rdd = rdd.flatMap(unpack_sent_vector)
    tf_df = tf_rdd.toDF(["query_id", "doc_id", "term_index", "TF"]) #query_id, sentence, term_index, TF

    #reduce by key --> query_id + term_index
    # get sentences with term frequency
    df_rdd = tf_rdd.map(lambda x: ((x[0],x[2]), 1)).reduceByKey(lambda a, b: a + b) #(query_id, term_index), sentence-term freq
    # make query_id the key
    df_rdd_flat = df_rdd.map(lambda x: (x[0][0], (x[0][1], x[1]))) #query_id, (term_index, sentence-term freq)

    #total number of sentences in each abstract
    #groupby number of query_id because each row have each sentence, groupby query give number of sentence in each query
    N_doc_rdd = tf_rdd.map(lambda x: ((x[0]), 1)).reduceByKey(lambda a, b: a + b)
    df_rdd_merge = df_rdd_flat.join(N_doc_rdd) #query_id, ((term_index, sentence-term freq), N)

    # get IDF
    idf_rdd = df_rdd_merge.map(lambda x: ((x[0],x[1][0][0]), math.log10(x[1][1]/x[1][0][1]))) # (query_id, term_index), IDF
    idf_df = idf_rdd.map(lambda x: (x[0][0], x[0][1], x[1])).toDF(["query_id","term_index","IDF"])

    # join TF and IDF dataframe
    # query_id, term_index, doc_id, TF, IDF
    tfidf_df = tf_df.join(idf_df, on = ['query_id', "term_index"], how ="inner")
    # calculate TF-IDF sum square per document
    tfidf_df = tfidf_df.withColumn("TFIDF_raw_Square", (((1+log10(col("TF"))))*col("IDF")) ** 2)
    window_spec = Window.partitionBy("doc_id")
    tfidf_df = tfidf_df.withColumn("SumOfSquares", sum(col("TFIDF_raw_Square")).over(window_spec))
    # calculate normalized TF-IDF
    tfidf_df = tfidf_df.withColumn("TFIDF", (((1+log10(col("TF"))))*col("IDF"))/col("SumOfSquares"))\
    .select("query_id","doc_id","term_index","TFIDF")

    # group tfidf and terms_index by doc_id (pivot by doc id, then concat columns(term_index) into list per doc)
    grouped_df_tfidf = tfidf_df.groupBy("query_id","doc_id").agg(collect_list("TFIDF").alias("TFIDF_List"))
    grouped_df_termID = tfidf_df.groupBy("query_id","doc_id").agg(collect_list("term_index").alias("ID_List")).withColumnRenamed("doc_id", "doc_id_1").withColumnRenamed("query_id", "query_id_1")
    grouped_df = grouped_df_tfidf.join(grouped_df_termID, grouped_df_tfidf.doc_id == grouped_df_termID.doc_id_1, "inner").select("query_id","doc_id","TFIDF_List","ID_List")
    # sort tfidf and term_indices for vectorization
    sorted_df = grouped_df.withColumn("Combined_List", arrays_zip(col("ID_List"), col("TFIDF_List"))) \
             .withColumn("Sorted_List", sort_array(col("Combined_List"), asc=True)) \
             .withColumn("Sorted_ID_List", expr("transform(Sorted_List, x -> x.ID_List)")) \
             .withColumn("Sorted_TFIDF_List", expr("transform(Sorted_List, x -> x.TFIDF_List)")) \
             .select("query_id","doc_id", "Sorted_ID_List", "Sorted_TFIDF_List")
    # vectorize
    sorted_df = sorted_df.withColumn("TFIDF_Vector", vectorize_tfidf_udf(sorted_df["Sorted_TFIDF_List"],sorted_df["Sorted_ID_List"]))\
        .select("query_id","doc_id", "TFIDF_Vector")

    return sorted_df


In [ ]:
merged_df.show()

+---------+----------------+------------------------+--------------------+--------------------+--------------------+
| query_id|    candidate_id|abstract_relevance_score| candidate_sentences|         tf_features|  query_TFIDF_Vector|
+---------+----------------+------------------------+--------------------+--------------------+--------------------+
|0704.0006|      1507.07518|        0.10591162797933|  The electron sp...|(262144,[2083,148...|(262144,[495,2325...|
|0704.0006|      1507.07518|        0.10591162797933|This type of magn...|(262144,[33387,40...|(262144,[495,2325...|
|0704.0006|      1507.07518|        0.10591162797933|A
resonance mode ...|(262144,[9781,202...|(262144,[495,2325...|
|0704.0117|cond-mat/0611254|     0.16276377377965698|  Phase diagram o...|(262144,[75750,78...|(262144,[1440,383...|
|0704.0117|cond-mat/0611254|     0.16276377377965698|In this form adap...|(262144,[82149,83...|(262144,[1440,383...|
|0704.0117|cond-mat/0611254|     0.16276377377965698|It is
shown

In [ ]:
sent_tfidf = get_sent_tfidfvector(merged_df.select("query_id","candidate_sentences","tf_features"))

In [ ]:
sent_tfidf = spark.createDataFrame(sent_tfidf.collect())

In [ ]:
sent_tfidf.show(2)

+---------+--------------------+--------------------+
| query_id|              doc_id|        TFIDF_Vector|
+---------+--------------------+--------------------+
|0704.0052|This equation res...|(262144,[50566,99...|
|0704.0052|This equation res...|(262144,[50566,99...|
+---------+--------------------+--------------------+
only showing top 2 rows



## 8 - Calculate Cosine similiarity for each sentence pair

In [ ]:
# rename columns before cross join
sent_tfidf = sent_tfidf.withColumnRenamed("doc_id","candidate_sentences")\
                                        .withColumnRenamed("TFIDF_Vector","candidate_sent_TFIDF_Vector")

# merge sentences_df with query/candidate_tfidf
merged_tfidf = merged_df.join(sent_tfidf, on = ["query_id", "candidate_sentences"], how = "inner")\
                        .select("query_id","candidate_id","abstract_relevance_score","candidate_sentences","query_TFIDF_Vector", "candidate_sent_TFIDF_Vector")

In [ ]:
merged_tfidf.show()

+---------+------------+------------------------+--------------------+--------------------+---------------------------+
| query_id|candidate_id|abstract_relevance_score| candidate_sentences|  query_TFIDF_Vector|candidate_sent_TFIDF_Vector|
+---------+------------+------------------------+--------------------+--------------------+---------------------------+
|0704.0030|  2107.04550|     0.12092555295496764|  Electronic flat...|(262144,[7096,919...|       (262144,[4254,116...|
|0901.0031|   1303.4880|     0.12481041882853254|We argue
that the...|(262144,[1590,446...|       (262144,[60022,10...|
|0704.0324|  1910.00059|     0.11356998948185043|In view of Greenf...|(262144,[3812,817...|       (262144,[2611,185...|
|0704.0001|   0903.3038|       0.241049511085364|In the present
wo...|(262144,[154,2931...|       (262144,[32817,34...|
|0704.0030|  2107.04550|     0.12092555295496764|Our results put f...|(262144,[7096,919...|       (262144,[14118,34...|
|0704.1267|  1612.06457|       0.0982354

In [ ]:
# Calculate cosine similarity for each combination of query and candidate documents
result_sent_data = merged_tfidf\
.withColumn("sentence_relevance_score", cos_sim_udf("query_TFIDF_Vector", "candidate_sent_TFIDF_Vector"))\
.select("query_id", "candidate_id","abstract_relevance_score", "candidate_sentences", "sentence_relevance_score")

In [ ]:
# Get candidate abstract with highest relevance to query abstract for each query abstract
# group by query id and order by cos_sim
groupbyID = Window.partitionBy("query_id","candidate_sentences").orderBy(F.desc("sentence_relevance_score"))
grouped_sent_result = result_sent_data.withColumn("grouped_index", F.row_number().over(groupbyID))
top_sent_result = grouped_sent_result.filter(F.col("grouped_index") == 1).drop("grouped_index")

In [ ]:
final_result = spark.createDataFrame(top_sent_result.collect())

In [ ]:
final_result.show()

+---------+------------+------------------------+--------------------+------------------------+
| query_id|candidate_id|abstract_relevance_score| candidate_sentences|sentence_relevance_score|
+---------+------------+------------------------+--------------------+------------------------+
|0704.0030|  2107.04550|     0.12092555295496764|  Electronic flat...|                     0.0|
|0901.0031|   1303.4880|     0.12481041882853254|We argue
that the...|                     0.0|
|0704.0324|  1910.00059|     0.11356998948185043|In view of Greenf...|    0.021966264332274704|
|0704.0001|   0903.3038|       0.241049511085364|In the present
wo...|      0.1569122423285437|
|0704.0030|  2107.04550|     0.12092555295496764|Our results put f...|     0.10396377843279975|
|0704.1267|  1612.06457|       0.098235446735887|However it is ver...|     0.04978401693928371|
|0704.0008|  1709.05526|     0.10325431242535459|In plastic deform...|     0.10124324284632691|
|0704.0030|  2107.04550|     0.120925552

## Test

In [ ]:
data1 = [(1, "A"), (2, "B")]
data2 = [(3, "X"), (4, "Y")]
df1 = spark.createDataFrame(data1, ["id1", "value1"])
df2 = spark.createDataFrame(data2, ["id2", "value2"])

# Perform a cross join
df3 = df1.crossJoin(df2)

In [ ]:
df1.show()

+---+------+
|id1|value1|
+---+------+
|  1|     A|
|  2|     B|
+---+------+



In [ ]:
df2.show()

+---+------+
|id2|value2|
+---+------+
|  3|     X|
|  4|     Y|
+---+------+



In [ ]:
df3.show()

+---+------+---+------+
|id1|value1|id2|value2|
+---+------+---+------+
|  1|     A|  3|     X|
|  1|     A|  4|     Y|
|  2|     B|  3|     X|
|  2|     B|  4|     Y|
+---+------+---+------+



In [ ]:
data = [(1, (100, [1, 2, 3], [23, 53, 24])),
        (2, (100, [2, 3, 4], [45, 3, 46])),
        (3, (100, [1, 3, 4], [1, 32, 45]))]

df = spark.createDataFrame(data, ["doc_id", "tf_features"])
df.show()

+------+--------------------+
|doc_id|         tf_features|
+------+--------------------+
|     1|{100, [1, 2, 3], ...|
|     2|{100, [2, 3, 4], ...|
|     3|{100, [1, 3, 4], ...|
+------+--------------------+



In [ ]:
rdd = df.rdd

In [ ]:
rdd.collect()

[Row(doc_id=1, tf_features=Row(_1=100, _2=[1, 2, 3], _3=[23, 53, 24])),
 Row(doc_id=2, tf_features=Row(_1=100, _2=[2, 3, 4], _3=[45, 3, 46])),
 Row(doc_id=3, tf_features=Row(_1=100, _2=[1, 3, 4], _3=[1, 32, 45]))]

In [ ]:
# stack
def calculate_tf(doc):
    doc_id, (num_features, term_indices, term_frequencies) = doc
    return [(doc_id, term_indices[i], term_frequencies[i]) for i in range(len(term_indices))]

tf_rdd = rdd.flatMap(calculate_tf)
tf_df = tf_rdd.toDF(["doc_index","term_index", "TF"])

In [ ]:
tf_rdd.collect()

[(1, 1, 23),
 (1, 2, 53),
 (1, 3, 24),
 (2, 2, 45),
 (2, 3, 3),
 (2, 4, 46),
 (3, 1, 1),
 (3, 3, 32),
 (3, 4, 45)]

In [ ]:
# calculate term-document frequency
df_rdd = tf_rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b)

In [ ]:
df_rdd.collect()

[(1, 2), (2, 2), (3, 3), (4, 2)]

In [ ]:
# calculate IDF
N = rdd.count()
idf_rdd = df_rdd.map(lambda x: (x[0], math.log10(N/(x[1]))))
idf_df = idf_rdd.toDF(["term_index_1", "IDF"])

In [ ]:
idf_rdd.collect()

[(1, 0.17609125905568124),
 (2, 0.17609125905568124),
 (3, 0.0),
 (4, 0.17609125905568124)]

In [ ]:
tfidf_df = tf_df.join(idf_df, tf_df.term_index == idf_df.term_index_1, "inner")

In [ ]:
tfidf_df.show()

+---------+----------+---+------------+-------------------+
|doc_index|term_index| TF|term_index_1|                IDF|
+---------+----------+---+------------+-------------------+
|        1|         1| 23|           1|0.17609125905568124|
|        3|         1|  1|           1|0.17609125905568124|
|        1|         3| 24|           3|                0.0|
|        2|         3|  3|           3|                0.0|
|        3|         3| 32|           3|                0.0|
|        1|         2| 53|           2|0.17609125905568124|
|        2|         2| 45|           2|0.17609125905568124|
|        2|         4| 46|           4|0.17609125905568124|
|        3|         4| 45|           4|0.17609125905568124|
+---------+----------+---+------------+-------------------+



In [ ]:
tfidf_df = tfidf_df.withColumn("TFIDF_raw_Square", (((1+log10(col("TF"))))*col("IDF")) ** 2)
window_spec = Window.partitionBy("doc_index")
tfidf_df = tfidf_df.withColumn("SumOfSquares", sum(col("TFIDF_raw_Square")).over(window_spec))

In [ ]:
tfidf_df = tfidf_df.withColumn("TFIDF", (((1+log10(col("TF"))))*col("IDF"))/col("SumOfSquares"))\
    .select("doc_index","term_index","TFIDF")

In [ ]:
tfidf_df.show()

+---------+----------+------------------+
|doc_index|term_index|             TFIDF|
+---------+----------+------------------+
|        1|         1|  1.03173340745023|
|        1|         3|               0.0|
|        1|         2| 1.190114450493661|
|        3|         1|0.7063682696231849|
|        3|         3|               0.0|
|        3|         4|1.8741451322980702|
|        2|         3|               0.0|
|        2|         2| 1.066344994032058|
|        2|         4|1.0701813252391914|
+---------+----------+------------------+



In [ ]:
grouped_df_tfidf = tfidf_df.groupBy("doc_index").agg(collect_list("TFIDF").alias("TFIDF_List"))
grouped_df_termID = tfidf_df.groupBy("doc_index").agg(collect_list("term_index").alias("ID_List")).withColumnRenamed("doc_index", "doc_index_1")
grouped_df = grouped_df_tfidf.join(grouped_df_termID, grouped_df_tfidf.doc_index == grouped_df_termID.doc_index_1, "inner").select("doc_index","TFIDF_List","ID_List")

In [ ]:
grouped_df.show(truncate = False)

+---------+---------------------------------------------+---------+
|doc_index|TFIDF_List                                   |ID_List  |
+---------+---------------------------------------------+---------+
|1        |[1.03173340745023, 0.0, 1.190114450493661]   |[1, 3, 2]|
|3        |[0.7063682696231849, 0.0, 1.8741451322980702]|[1, 3, 4]|
|2        |[0.0, 1.066344994032058, 1.0701813252391914] |[3, 2, 4]|
+---------+---------------------------------------------+---------+



In [ ]:
sorted_df = grouped_df.withColumn("Combined_List", arrays_zip(col("ID_List"), col("TFIDF_List"))) \
             .withColumn("Sorted_List", sort_array(col("Combined_List"), asc=True)) \
             .withColumn("Sorted_ID_List", expr("transform(Sorted_List, x -> x.ID_List)")) \
             .withColumn("Sorted_TFIDF_List", expr("transform(Sorted_List, x -> x.TFIDF_List)")) \
             .select("doc_index", "Sorted_ID_List", "Sorted_TFIDF_List")

In [ ]:
sorted_df.show(truncate = False)

+---------+--------------+---------------------------------------------+
|doc_index|Sorted_ID_List|Sorted_TFIDF_List                            |
+---------+--------------+---------------------------------------------+
|1        |[1, 2, 3]     |[1.03173340745023, 1.190114450493661, 0.0]   |
|3        |[1, 3, 4]     |[0.7063682696231849, 0.0, 1.8741451322980702]|
|2        |[2, 3, 4]     |[1.066344994032058, 0.0, 1.0701813252391914] |
+---------+--------------+---------------------------------------------+



In [ ]:
def vectorize(values, index):
    return Vectors.sparse(size, index, values)
vectorize_tfidf_udf = udf(vectorize, VectorUDT())

sorted_df = sorted_df.withColumn("TFIDF_Vector", vectorize_tfidf_udf(sorted_df["Sorted_TFIDF_List"],sorted_df["Sorted_ID_List"]))\
    .select("doc_index", "TFIDF_Vector")

In [ ]:
sorted_df.show(truncate = False)

+---------+------------------------------------------------------------+
|doc_index|TFIDF_Vector                                                |
+---------+------------------------------------------------------------+
|1        |(262144,[1,2,3],[1.03173340745023,1.190114450493661,0.0])   |
|3        |(262144,[1,3,4],[0.7063682696231849,0.0,1.8741451322980702])|
|2        |(262144,[2,3,4],[1.066344994032058,0.0,1.0701813252391914]) |
+---------+------------------------------------------------------------+



## Method 2

## 5 - Calculate TF-IDF

In [ ]:
# Calculate TF-IDF (Inverse Document Frequency)
# Union queries and candidates then drop duplicates
idf = IDF(inputCol="tf_features", outputCol="tfidf_features")
idf_model = idf.fit(query_featurised.union(candidate_featurised).dropDuplicates())

#the tfidf_features output is in the form: (numFeatures,[a1,a2,a3,a4],[b1,b2,b3,b4])
# where "a" is the hash-index and "b" is the TF-IDF value
# TF-IDF = number of term t in doc1 * ln((total documents + 1)/(number of documents with term t)))
query_tfidf = idf_model.transform(query_featurised)
candidate_tfidf = idf_model.transform(candidate_featurised)

In [ ]:
def normalize_tfidf(tfidf_vector):
    #convert to base 10 log from natural log
    tfidf_values = [val/math.log(10) for val in tfidf_vector.values]
    tfidf_vector_log10 = Vectors.sparse(tfidf_vector.size, tfidf_vector.indices, tfidf_values)
    sum_of_squares = float(tfidf_vector_log10.dot(tfidf_vector_log10))
    if sum_of_squares == 0:
        return Vectors.sparse(len(tfidf_vector_log10), [])
    normalized_values = [val/(sum_of_squares**0.5) for val in tfidf_vector_log10.values]
    normalized_vector = Vectors.sparse(tfidf_vector_log10.size, tfidf_vector_log10.indices, normalized_values)
    return normalized_vector

#return vector data-type
normalize_tfidf_udf = udf(normalize_tfidf, VectorUDT())

In [ ]:
# Normalize the TF-IDF vectors
query_norm = query_tfidf.withColumn("normalized_tfidf", normalize_tfidf_udf("tfidf_features"))
candidate_norm = query_tfidf.withColumn("normalized_tfidf", normalize_tfidf_udf("tfidf_features"))

In [ ]:
query_tfidf.tfidf_features.size

Column<'tfidf_features[size]'>

## 6 - Calculate relevance (cosine similarity)

In [ ]:
def cosine_sim(v1, v2):
    dot_product = float(v1.dot(v2))
    #L2 Norm -> sum of squares
    L2norm_v1 = float(v1.norm(2))
    L2norm_v2 = float(v2.norm(2))
    cos_sim = dot_product / (L2norm_v1 * L2norm_v2)
    return cos_sim

cos_sim_udf = udf(cosine_sim, DoubleType())

In [ ]:
# Normalize the TF-IDF vectors
query_norm = query_tfidf.withColumn("normalized_tfidf_q", normalize_tfidf_udf("tfidf_features"))
candidate_norm = query_tfidf.withColumn("normalized_tfidf_c", normalize_tfidf_udf("tfidf_features"))

# rename columns before cross join
query_norm = query_norm.withColumnRenamed("id","query_id")\
    .withColumnRenamed("abstract","query_abstract")
candidate_norm = candidate_norm.withColumnRenamed("id","candidate_id")\
    .withColumnRenamed("abstract","candidate_abstract")

In [ ]:
# Calculate cosine similarity for each combination of query and candidate documents
result_data = query_norm.crossJoin(candidate_norm)\
.withColumn("relevance", cos_sim_udf("normalized_tfidf_q", "normalized_tfidf_c"))\
.select("query_id", "candidate_id", "relevance", "query_abstract", "candidate_abstract")

# remove rows joining to themselves
result_data = result_data.filter(col("query_id") != col("candidate_id"))

## 7 - Get most relevant candidate for each query

In [ ]:
# Get candidate abstract with highest relevance to query abstract for each query abstract
# group by query id and order by cos_sim
groupbyID = Window.partitionBy("query_id").orderBy(F.desc("relevance"))
grouped_result = result_data.withColumn("grouped_index", F.row_number().over(groupbyID))
top_result = grouped_result.filter(F.col("grouped_index") == 1).drop("grouped_index")

## 8 - Get most matching sentences

In [ ]:
def split_sent(block):
    sentences = nltk.sent_tokenize(block)
    return sentences

split_sent_udf = udf(split_sent, ArrayType(StringType()))

In [ ]:
# split into sentences
top_result = top_result.withColumn("query_sentences", split_sent_udf("query_abstract"))
top_result = top_result.withColumn("candidate_sentences", split_sent_udf("candidate_abstract"))

# union the sentences and keep duplicates
top_result = top_result.withColumn("all_sentences", F.array_union("query_sentences", "candidate_sentences"))

In [ ]:
# split each of the sentences into words using expr function
top_result_words = top_result.withColumn("words", preprocess_udf("all_sentences"))

In [ ]:
# tfidf_vector = query_tfidf.select("tfidf_features").collect()
# tfidf_vector = [row.tfidf_features for row in tfidf_vector]
# query_tfidf.select("tfidf_features").show(1,truncate = False)
# query_tfidf.show(1, truncate = False)
# query_norm.select("normalized_tfidf").show(1,truncate = False)
# query_norm.select("tfidf_features").show(1,truncate = False)

In [ ]:
top_result.select("all_sentences").show(1,truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------